<a href="https://colab.research.google.com/github/slothfulz/Pair-Trading-Backtester/blob/main/cointegration_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from itertools import combinations

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', storage_options={'User-Agent': 'Mozilla/5.0'})


In [4]:
df = yf.download(tickers[0]['Symbol'].tolist(), start = '2023-01-01')

/tmp/ipython-input-4132775730.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers[0]['Symbol'].tolist(), start = '2023-01-01')
[*********************100%***********************]  503 of 503 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-01-01 -> 2025-12-03)')


In [5]:
df

Price      Adj Close             Close                                      \
Ticker          BF.B BRK.B           A        AAPL        ABBV        ABNB   
Date                                                                         
2023-01-03       NaN   NaN  147.012192  123.211205  145.483810   84.900002   
2023-01-04       NaN   NaN  148.609283  124.482033  146.657501   88.720001   
2023-01-05       NaN   NaN  149.040436  123.161949  146.478317   87.709999   
2023-01-06       NaN   NaN  144.690002  127.693588  149.219925   88.519997   
2023-01-09       NaN   NaN  144.494064  128.215698  144.838730   89.239998   
...              ...   ...         ...         ...         ...         ...   
2025-11-26       NaN   NaN  154.369995  277.549988  227.660004  116.739998   
2025-11-28       NaN   NaN  153.500000  278.850006  227.699997  116.989998   
2025-12-01       NaN   NaN  150.100006  283.100006  225.110001  118.800003   
2025-12-02       NaN   NaN  148.809998  286.190002  224.369995  118.500000   
2025-12-03       NaN   NaN  149.259995  284.149994  230.240005  120.129997   

Price                                                      ...   Volume  \
Ticker             ABT       ACGL         ACN        ADBE  ...       WY   
Date                                                       ...            
2023-01-03  103.370216  59.393120  258.138824  336.920013  ...  2939100   
2023-01-04  104.907837  59.687901  257.260101  341.410004  ...  3287700   
2023-01-05  104.521072  59.849552  251.185379  328.440002  ...  2976200   
2023-01-06  105.964371  60.762417  257.135956  332.750000  ...  2890600   
2023-01-09  105.794548  59.592808  261.472351  341.980011  ...  2511100   
...                ...        ...         ...         ...  ...      ...   
2025-11-26  128.539993  94.400002  247.850006  317.519989  ...  5439500   
2025-11-28  128.899994  93.919998  250.000000  320.130005  ...  3648900   
2025-12-01  128.009995  93.160004  257.429993  322.850006  ...  7107800   
2025-12-02  126.320000  92.690002  261.019989  322.809998  ...  5232900   
2025-12-03  125.290001  92.720001  272.850006  326.779999  ...  3941574   

Price                                                                        \
Ticker         WYNN      XEL       XOM      XYL       XYZ      YUM      ZBH   
Date                                                                          
2023-01-03  3318600  2684500  15146200   812400  16416500  2307100   921100   
2023-01-04  5743600  2265900  18058400   905900  11749500  1139400  1389400   
2023-01-05  2983000  2104500  15946600  1295700  11336200  1078200  2238300   
2023-01-06  2901900  2976200  16348100   636200  15529600  1559400  2348900   
2023-01-09  2751000  2707800  17964600  1224700  12987600  1211200  1423600   
...             ...      ...       ...      ...       ...      ...      ...   
2025-11-26  1946600  2850700  11650600  1442600   6770500  1833300  1734900   
2025-11-28   861200  1364400   7240800  1554300   3459900   923300   890000   
2025-12-01  2831000  3885400  12658100  1395700   8119900  3536400  2594200   
2025-12-02  1215100  4616700  14607700  1191600  20737700  1781300  1536400   
2025-12-03  1119368  3440321  11782965   826015  11390012  1834300  1699342   

Price                        
Ticker        ZBRA      ZTS  
Date                         
2023-01-03  295400  1840600  
2023-01-04  311600  1796100  
2023-01-05  368200  1560900  
2023-01-06  552700  1761200  
2023-01-09  535400  1700500  
...            ...      ...  
2025-11-26  507700  3199000  
2025-11-28  206400  2143000  
2025-12-01  475800  4206900  
2025-12-02  486200  6896900  
2025-12-03  573945  4038314  

[733 rows x 2517 columns]

In [6]:
combis = pd.DataFrame(combinations(df.columns,2))

In [7]:
combis.columns = ['1st_stock', '2nd_stock']

In [8]:
corr = df.corr()

combis['correlations'] = combis.apply(
    lambda r: corr.loc[r['1st_stock'], r['2nd_stock']],
    axis=1
)

KeyboardInterrupt: 

Filtering for highly correlated stock pairs from 2023

In [ ]:
combis['correlations'] = combis.apply(lambda row: np.corrcoef(df[row['1st_stock']], df[row['2nd_stock']])[0,1], axis =1)

KeyboardInterrupt: 

In [1]:
combis = combis[combis['correlations'] > 0.95]

NameError: name 'combis' is not defined

ADF test to check relatively stationary pairs

In [ ]:
def do_reg(stock1, stock2):
  x=df[stock1].values
  y=df[stock2].values
  X= sm.add_constant(x)
  model = sm.OLS(y,X).fit()
  alpha,beta = model.params
  residuals = y-(alpha + beta * X[:,1])
  if np.isna(residuals).any():
    return np.nan

  adf_result = adfuller(residuals)
  return adf_result[0], adf_result[1], beta, alpha



In [ ]:
combis[['adf_stat','p_value', 'beta','alpha']] = combis.apply(lambda row:   pd.Series(do_reg(row['1st_stock'], row['2nd_stock'])), axis = 1)

In [ ]:
combis[combis.p_value<0.01].sort_values(by = 'adf_stat')

In [ ]:
spread = df['']-()